# Discriminación Gamma/Neutrón basada en ML

Flujo de trabajo basado en R. S. Molina, I. R. Morales, M. L. Crespo, V. G. Costa, S. Carrato y G. Ramponi, "An End-to-End Workflow to Efficiently Compress and Deploy DNN Classifiers on SoC/FPGA", en IEEE Embedded Systems Letters, vol. 16, no. 3, pp. 255-258, septiembre 2024, doi: 10.1109/LES.2023.3343030.

Código adaptado del repositorio oficial de "An End-to-End Workflow to Efficiently Compress and Deploy DNN Classifiers on SoC/FPGA".

Uso de dataset abierto disponible en: https://doi.org/10.5281/zenodo.8037058

Documentación de hls4ml: https://fastmachinelearning.org/hls4ml/

--- 
### Librerias

In [1]:
import os
import numpy as np
import matplotlib.pyplot as plt

import tensorflow as tf
from tensorflow.keras.models import *
from tensorflow.keras.layers import *
from qkeras import *
from qkeras import QActivation
from qkeras import QDense, QConv2DBatchnorm

import hls4ml

# Ruta hacia Vitis HLS

El directorio de Vitis HLS debe especificarse para poder usar `hls4ml` junto con la herramienta de high-level synthesis (síntesis de alto nivel).

In [2]:
# PATH Xilinx Virtual Machine
os.environ['PATH'] = '/mnt/d/Xilinx/Vitis_HLS/2022.2/bin:' + os.environ['PATH']
os.environ['PATH'] = '/mnt/d/Xilinx/Vitis/2022.2/bin:' + os.environ['PATH']
os.environ['PATH'] = '/mnt/d/Xilinx/Vivado/2022.2/bin:' + os.environ['PATH']

## Cargar el modelo

Se carga el modelo del estudiante para hacer la conversion

In [3]:
# Load keras model from file

from qkeras.utils import _add_supported_quantized_objects

co = {}
_add_supported_quantized_objects(co)

model = load_model('./models/studentModel_GN_GICM.h5', custom_objects=co)
model.summary()

Model: "studentMLP"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 fc1 (QDense)                (None, 6)                 972       
                                                                 
 relu0 (QActivation)         (None, 6)                 0         
                                                                 
 dropout_8 (Dropout)         (None, 6)                 0         
                                                                 
 fc2 (QDense)                (None, 4)                 28        
                                                                 
 relu1 (QActivation)         (None, 4)                 0         
                                                                 
 dropout_9 (Dropout)         (None, 4)                 0         
                                                                 
 fc3 (QDense)                (None, 2)                 1

### Integracion con hls4ml

hls4ml es un paquete de Python desarrollado para convertir modelos de machine learning (**ML**) en proyectos de HLS (**High-Level Synthesis**), lo que permite desplegar inferencias basadas en ML en hardware como **FPGAs**. Más detalles se pueden encontrar en la documentación de hls4ml.

El usuario puede controlar varias opciones relacionadas con el modelo, incluyendo:

- **Precisión**: Permite definir la precisión de los cálculos en el modelo, por ejemplo, usando representación de punto fijo (fixed-point) o punto flotante (floating-point).

- **Flujo de datos / Reutilización de recursos**: Controla el nivel de paralelismo o streaming en la implementación del modelo, permitiendo distintos grados de pipelining.

- **Quantization Aware Training (QAT)**: Permite lograr un rendimiento optimizado con baja precisión utilizando herramientas como **QKeras**. Los modelos entrenados con QKeras se benefician automáticamente del análisis que hace **hls4ml** de los modelos **QKeras** durante la inferencia.

Para implementar un modelo en FPGA, se debe crear una configuración HLS usando la función:

`hls4ml.utils.config_from_keras_model(kerasModel, granularity)`

donde:

kerasModel es el modelo preentrenado que se quiere implementar en FPGA.

granularity determina el nivel de configuración, y puede tomar dos valores:

'model': La misma configuración se aplica a todo el modelo (por ejemplo, todas las capas usan precisión de 16 bits en punto fijo).

'name': Se pueden aplicar configuraciones específicas por capa (por ejemplo, la capa de entrada en 8 bits punto fijo, mientras que la segunda capa en 16 bits punto fijo).

In [4]:
hls_config = hls4ml.utils.config_from_keras_model(model, granularity='name')


Interpreting Sequential
Topology:
Layer name: inputLayer, layer type: InputLayer, input shapes: [[None, 161]], output shape: [None, 161]
Layer name: fc1, layer type: QDense, input shapes: [[None, 161]], output shape: [None, 6]
Layer name: relu0, layer type: Activation, input shapes: [[None, 6]], output shape: [None, 6]
Layer name: fc2, layer type: QDense, input shapes: [[None, 6]], output shape: [None, 4]
Layer name: relu1, layer type: Activation, input shapes: [[None, 4]], output shape: [None, 4]
Layer name: fc3, layer type: QDense, input shapes: [[None, 4]], output shape: [None, 2]
Layer name: relu2, layer type: Activation, input shapes: [[None, 2]], output shape: [None, 2]
Layer name: fc4, layer type: QDense, input shapes: [[None, 2]], output shape: [None, 4]
Layer name: relu3, layer type: Activation, input shapes: [[None, 4]], output shape: [None, 4]
Layer name: fc5, layer type: QDense, input shapes: [[None, 4]], output shape: [None, 3]
Layer name: relu4, layer type: Activation, in

In [5]:
from src import plotting

print("-----------------------------------")
plotting.print_dict(hls_config)
print("-----------------------------------")

-----------------------------------
Model
  Precision
    default:         fixed<16,6>
  ReuseFactor:       1
  Strategy:          Latency
  BramFactor:        1000000000
  TraceOutput:       False
LayerName
  inputLayer
    Trace:           False
    Precision
      result:        auto
  fc1
    Trace:           False
    Precision
      result:        auto
      weight:        fixed<8,5,TRN,WRAP,0>
      bias:          fixed<8,5,TRN,WRAP,0>
  fc1_linear
    Trace:           False
    Precision
      result:        auto
  relu0
    Trace:           False
    Precision
      result:        fixed<8,1,RND_CONV,SAT,0>
  fc2
    Trace:           False
    Precision
      result:        auto
      weight:        fixed<8,5,TRN,WRAP,0>
      bias:          fixed<8,5,TRN,WRAP,0>
  fc2_linear
    Trace:           False
    Precision
      result:        auto
  relu1
    Trace:           False
    Precision
      result:        fixed<8,1,RND_CONV,SAT,0>
  fc3
    Trace:           False
    Preci

In [6]:
for Layer in hls_config['LayerName'].keys():
    hls_config['LayerName'][Layer]['Strategy'] = 'Latency'
    hls_config['LayerName'][Layer]['ReuseFactor'] = 1
    hls_config['LayerName'][Layer]['Precision'] = 'ap_fixed<8,4>'

hls_config['LayerName']['outputActivation']['Strategy'] = 'Stable'
hls_config['Model']['Precision'] = 'ap_fixed<16,6>'

hls_config['LayerName']['inputLayer']['Precision'] = 'ap_fixed<16,6>'

###  hls4ml con Vitis HLS como backend

---

Se debe correr la celda, saldra un error, pero al final es necesario ejecutar en la terminal lo siguiente:


``` bash
$env:PATH += ";D:\Xilinx\Vitis_HLS\2022.2\bin"
vitis_hls -version
cd .\pythonModel\hlsPrj\
vitis_hls -f .\build_prj.tcl
```

In [7]:
# Create configuration for Vitis HLS as backend.
cfg = hls4ml.converters.create_config(backend='Vitis')

# HLSConfig correspond to the configuration created in hls_config 
cfg['HLSConfig']  = hls_config
# Model to be converted
cfg['KerasModel'] = model
# Folder where the HLS project will be created
cfg['OutputDir']  = './hlsPrj/'
# FPGA part 
cfg['Part'] = 'xc7z020clg484-1'  # PYNQ-Z1 or Zedboard: xc7z020clg484-1  
  
hls_model = hls4ml.converters.keras_to_hls(cfg)

hls_model.compile()
### Usar en terminal el sigueinte comando
# $env:PATH += ";D:\Xilinx\Vitis_HLS\2022.2\bin"
# vitis_hls -version
# cd .\pythonModel\hlsPrj\
# vitis_hls -f .\build_prj.tcl

Interpreting Sequential
Topology:
Layer name: inputLayer, layer type: InputLayer, input shapes: [[None, 161]], output shape: [None, 161]
Layer name: fc1, layer type: QDense, input shapes: [[None, 161]], output shape: [None, 6]
Layer name: relu0, layer type: Activation, input shapes: [[None, 6]], output shape: [None, 6]
Layer name: fc2, layer type: QDense, input shapes: [[None, 6]], output shape: [None, 4]
Layer name: relu1, layer type: Activation, input shapes: [[None, 4]], output shape: [None, 4]
Layer name: fc3, layer type: QDense, input shapes: [[None, 4]], output shape: [None, 2]
Layer name: relu2, layer type: Activation, input shapes: [[None, 2]], output shape: [None, 2]
Layer name: fc4, layer type: QDense, input shapes: [[None, 2]], output shape: [None, 4]
Layer name: relu3, layer type: Activation, input shapes: [[None, 4]], output shape: [None, 4]
Layer name: fc5, layer type: QDense, input shapes: [[None, 4]], output shape: [None, 3]
Layer name: relu4, layer type: Activation, in

Done
"." no se reconoce como un comando interno o externo,
programa o archivo por lotes ejecutable.



Exception: Failed to compile project "myproject"

In [9]:
 # This will perform the synthesis of the HLS project, showing the main results (latency and resource usage) once the process is completed. 

hls_model.build(csim=False, export=False)

Vivado synthesis report not found.
Implementation report not found.
Timing report not found.


{'CSynthesisReport': {'TargetClockPeriod': '5.00',
  'EstimatedClockPeriod': '3.554',
  'BestLatency': '31',
  'WorstLatency': '31',
  'IntervalMin': '1',
  'IntervalMax': '1',
  'BRAM_18K': '1',
  'DSP': '5',
  'FF': '15288',
  'LUT': '15380',
  'URAM': '0',
  'AvailableBRAM_18K': '280',
  'AvailableDSP': '220',
  'AvailableFF': '106400',
  'AvailableLUT': '53200',
  'AvailableURAM': '0'},
 'CosimReport': {'RTL': 'Verilog',
  'Status': 'Pass',
  'LatencyMin': 31,
  'LatencyMax': 31,
  'IntervalMin': 1,
  'IntervalMax': 1,
  'LatencyAvg': 31.0,
  'IntervalAvg': 1.0}}